In [6]:
import pandas as pd
import numpy as np
from astropy.io import fits
from tqdm import tqdm
import os

In [7]:
def convertConstDat():
    folder = 'matlab/const_models_2sigma_hull'
    files = os.listdir(folder)
    fneg = []
    ffast = []
    z = []
    co = []

    for f in tqdm(files):
        if f[-4:] == '.csv':
            tags = f[:-4].split('_')
            fneg.append(float(tags[2]))
            ffast.append(float(tags[3]))

            data = pd.read_csv(folder+'/'+f)
            z.append(data['z'].values)
            co.append(data['co14_co'].values)

    depth = np.array(z[0])
    co14 = np.array(co)
    fneg = np.array(fneg)
    ffast = np.array(ffast)

    depthHDU = fits.PrimaryHDU(data=depth)
    depthHDU.header['EXTNAME'] = 'DEPTH'
    depthHDU.header['BUNIT'] = 'meter'

    co14HDU = fits.ImageHDU(data=co14)
    co14HDU.header['EXTNAME'] = 'CO14'
    co14HDU.header['BUNIT'] = '1/gram'

    col2 = fits.Column(name='FOMUNEG', format='D', array=fneg)
    col3 = fits.Column(name='FOMUFAST', format='D', array=ffast)
    cols = fits.ColDefs([col2,col3])
    fofactorsHDU = fits.BinTableHDU.from_columns(cols)
    fofactorsHDU.header['EXTNAME'] = 'FOFACTOR'
    fofactorsHDU.header['N_FONEG'] = len(np.unique(fneg))
    fofactorsHDU.header['N_FOFAST'] = len(np.unique(ffast))
    fofactorsHDU.header['FNEGMIN'] = min(fneg)
    fofactorsHDU.header['FNEGMAX'] = max(fneg)
    fofactorsHDU.header['DELTNEG'] = np.diff(np.sort(np.unique(fneg)))[0] if len(np.unique(fneg))>1 else 0.
    fofactorsHDU.header['FFASTMIN'] = min(ffast)
    fofactorsHDU.header['FFASTMAX'] = max(ffast)
    fofactorsHDU.header['DELTFAST'] = np.diff(np.sort(np.unique(ffast)))[0] if len(np.unique(ffast))>1 else 0.

    hdul = fits.HDUList([depthHDU, co14HDU, fofactorsHDU])

    outfile = 'models/balco_14co_const_models.fits'
    hdul.writeto(outfile)

In [8]:
def convertVarDat(style='linear', fixed='past', f_factors='const'):
    folder = 'matlab/{}_models_{}'.format(style, fixed)
    files = os.listdir(folder)
    amp = []
    fneg = []
    ffast = []
    z = []
    co = []

    for f in tqdm(files):
        if f[-4:] == '.csv':
            tags = f[:-4].split('_')
            if f_factors=='all' or (float(tags[3])==0.066 and float(tags[4])==0.072):
                amp.append(float(tags[2]))
                fneg.append(float(tags[3]))
                ffast.append(float(tags[4]))

                data = pd.read_csv(folder+'/'+f)
                z.append(data['z'].values)
                co.append(data['co14_co'].values)

    depth = np.array(z[0])
    co14 = np.array(co)
    ampl = np.array(amp)
    fneg = np.array(fneg)
    ffast = np.array(ffast)

    depthHDU = fits.PrimaryHDU(data=depth)
    depthHDU.header['EXTNAME'] = 'DEPTH'
    depthHDU.header['BUNIT'] = 'meter'

    co14HDU = fits.ImageHDU(data=co14)
    co14HDU.header['EXTNAME'] = 'CO14'
    co14HDU.header['BUNIT'] = '1/gram'

    col1 = fits.Column(name='AMPL', format='D', array=ampl)
    cols = fits.ColDefs([col1])
    amplHDU = fits.BinTableHDU.from_columns(cols)
    amplHDU.header['EXTNAME'] = 'AMPL'
    amplHDU.header['N_AMPL'] = len(np.unique(ampl))
    amplHDU.header['AMPLMIN'] = min(ampl)
    amplHDU.header['AMPLMAX'] = max(ampl)
    amplHDU.header['DELTAMPL'] = np.diff(np.sort(np.unique(ampl)))[0] if len(np.unique(ampl))>1 else 0.

    col2 = fits.Column(name='FOMUNEG', format='D', array=fneg)
    col3 = fits.Column(name='FOMUFAST', format='D', array=ffast)
    cols = fits.ColDefs([col2,col3])
    fofactorsHDU = fits.BinTableHDU.from_columns(cols)
    fofactorsHDU.header['EXTNAME'] = 'FOFACTOR'
    fofactorsHDU.header['N_FONEG'] = len(np.unique(fneg))
    fofactorsHDU.header['N_FOFAST'] = len(np.unique(ffast))
    fofactorsHDU.header['FNEGMIN'] = min(fneg)
    fofactorsHDU.header['FNEGMAX'] = max(fneg)
    fofactorsHDU.header['DELTNEG'] = np.diff(np.sort(np.unique(fneg)))[0] if len(np.unique(fneg))>1 else 0.
    fofactorsHDU.header['FFASTMIN'] = min(ffast)
    fofactorsHDU.header['FFASTMAX'] = max(ffast)
    fofactorsHDU.header['DELTFAST'] = np.diff(np.sort(np.unique(ffast)))[0] if len(np.unique(ffast))>1 else 0.


    hdul = fits.HDUList([depthHDU, co14HDU, amplHDU, fofactorsHDU])
    
    if style=='burst':
        outfile = 'models/balco_14co_burst_models_{}.fits'.format(f_factors)
    else:
        outfile = 'models/balco_14co_{}_models_{}_{}.fits'.format(style, fixed, f_factors)
    hdul.writeto(outfile)

In [9]:
def convertDeltaDat(style='neg'):
    folder = 'matlab/delta_{}_models'.format(style)
    files = os.listdir(folder)
    t = []
    fneg = []
    ffast = []
    z = []
    co = []

    for f in tqdm(files):
        if f[-4:] == '.csv':
            tags = f[:-4].split('_')
            t.append(float(tags[2]))
            fneg.append(float(tags[3]))
            ffast.append(float(tags[4]))

            data = pd.read_csv(folder+'/'+f)
            z.append(data['z'].values)
            co.append(data['co14_co'].values)

    depth = np.array(z[0])
    co14 = np.array(co)
    t_spike = np.array(t)
    fneg = np.array(fneg)
    ffast = np.array(ffast)

    depthHDU = fits.PrimaryHDU(data=depth)
    depthHDU.header['EXTNAME'] = 'DEPTH'
    depthHDU.header['BUNIT'] = 'meter'

    co14HDU = fits.ImageHDU(data=co14)
    co14HDU.header['EXTNAME'] = 'CO14'
    co14HDU.header['BUNIT'] = '1/gram'

    col1 = fits.Column(name='T_SPIKE', format='D', array=t_spike)
    cols = fits.ColDefs([col1])
    timeHDU = fits.BinTableHDU.from_columns(cols)
    timeHDU.header['EXTNAME'] = 'T_SPIKE'
    timeHDU.header['N_TIME'] = len(np.unique(t_spike))
    timeHDU.header['T_MIN'] = min(t_spike)
    timeHDU.header['T_MAX'] = max(t_spike)
    timeHDU.header['DELTA_T'] = np.diff(np.sort(np.unique(t_spike)))[0] if len(np.unique(t_spike))>1 else 0.

    col2 = fits.Column(name='FOMUNEG', format='D', array=fneg)
    col3 = fits.Column(name='FOMUFAST', format='D', array=ffast)
    cols = fits.ColDefs([col2,col3])
    fofactorsHDU = fits.BinTableHDU.from_columns(cols)
    fofactorsHDU.header['EXTNAME'] = 'FOFACTOR'
    fofactorsHDU.header['N_FONEG'] = len(np.unique(fneg))
    fofactorsHDU.header['N_FOFAST'] = len(np.unique(ffast))
    fofactorsHDU.header['FNEGMIN'] = min(fneg)
    fofactorsHDU.header['FNEGMAX'] = max(fneg)
    fofactorsHDU.header['DELTNEG'] = np.diff(np.sort(np.unique(fneg)))[0] if len(np.unique(fneg))>1 else 0.
    fofactorsHDU.header['FFASTMIN'] = min(ffast)
    fofactorsHDU.header['FFASTMAX'] = max(ffast)
    fofactorsHDU.header['DELTFAST'] = np.diff(np.sort(np.unique(ffast)))[0] if len(np.unique(ffast))>1 else 0.


    hdul = fits.HDUList([depthHDU, co14HDU, timeHDU, fofactorsHDU])

    outfile = 'models/balco_14co_delta_{}_models.fits'.format(style)
    hdul.writeto(outfile)

In [10]:
convertConstDat()

100%|████████████████████████████████████████████████████████████████████████████████| 118/118 [00:01<00:00, 78.25it/s]


OSError: File 'FITS_models/balco_14co_const_models.fits' already exists.

In [ ]:
for x in ['linear','step']:
    for y in ['past','pres']:
        for z in ['const','all']:
            convertVarDat(x,y,z)

In [ ]:
for x in ['const', 'all']:
    convertVarDat('burst', f_factors=x)

In [ ]:
for x in ['neg','fast']:
    convertDeltaDat(x)